In [1]:
import pandas as pd
import requests
import itertools
from bs4 import BeautifulSoup
from requests.exceptions import ChunkedEncodingError

In [2]:

# Read the URLs from the text file
with open('all_recipes_url.txt', 'r') as file:
    all_recipes_url = file.readlines()

# Strip newline characters from each URL
all_recipes_url = [url.strip() for url in all_recipes_url]

len(all_recipes_url)

360

In [3]:
all_recipes_url

['https://www.halfbakedharvest.com/sour-cream-and-onion-smashed-potatoes/',
 'https://www.halfbakedharvest.com/tomato-cheese-pie/',
 'https://www.halfbakedharvest.com/open-face-tomato-bacon-sandwich/',
 'https://www.halfbakedharvest.com/garlic-oil-feta-dip/',
 'https://www.halfbakedharvest.com/marinated-burrata/',
 'https://www.halfbakedharvest.com/pull-apart-cheeseburger-sliders/',
 'https://www.halfbakedharvest.com/31-recipes-to-cook-in-july/',
 'https://www.halfbakedharvest.com/watermelon-peach-salad/',
 'https://www.halfbakedharvest.com/7-layer-dip/',
 'https://www.halfbakedharvest.com/honey-garlic-parmesan-biscuits/',
 'https://www.halfbakedharvest.com/bbq-snack-mix/',
 'https://www.halfbakedharvest.com/tomato-burrata-caprese/',
 'https://www.halfbakedharvest.com/31-recipes-to-cook-in-may/',
 'https://www.halfbakedharvest.com/garlic-olive-oil-dip/',
 'https://www.halfbakedharvest.com/garlic-parmesan-boneless-wings/',
 'https://www.halfbakedharvest.com/fried-goat-cheese-balls/',
 '

# Extract Information of all recipes

In [4]:
failed_url = []

In [5]:
# Empty Dataframe
columns = [
    'title',
    'prep_time',
    'cook_time',
    'total_time',
    'ingredients',
    'ingredients with measurement',
    'instructions',
    'calories',
    'servings',
    'images'
]

In [6]:
final = pd.DataFrame(columns=columns)
final

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images


# Extract Function

In [7]:
def extract(soup):
    try:
        # TITLE
        h2_tag = soup.find('h2', class_='wprm-recipe-name')  # Replaced temp_soup with soup
        if h2_tag:
            title.append(h2_tag.text.strip())
        else:
            title.append('-')

        # PREP TIME
        prep_time_elem = soup.find(class_='wprm-recipe-prep-time-container')  # Replaced temp_soup with soup
        if prep_time_elem:
            time_dict = {}
            hours_tag = prep_time_elem.find('span', class_='wprm-recipe-prep_time-hours')
            minutes_tag = prep_time_elem.find('span', class_='wprm-recipe-prep_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value
            prep_time.append(time_dict) 
        else:
            prep_time.append('-')

        # COOK TIME
        cook_time_elem = soup.find(class_='wprm-recipe-cook-time-container')  # Replaced temp_soup with soup
        if cook_time_elem:
            time_dict = {}
            hours_tag = cook_time_elem.find('span', class_='wprm-recipe-cook_time-hours')
            minutes_tag = cook_time_elem.find('span', class_='wprm-recipe-cook_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value

            cook_time.append(time_dict) 
        else:
            cook_time.append('-')

        # TOTAL TIME
        total_time_elem = soup.find(class_='wprm-recipe-total-time-container')  # Replaced temp_soup with soup
        if total_time_elem:
            time_dict = {}
            hours_tag = total_time_elem.find('span', class_='wprm-recipe-total_time-hours')
            minutes_tag = total_time_elem.find('span', class_='wprm-recipe-total_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value
            total_time.append(time_dict) 
        else:
            total_time.append('-')

        # INGREDIENTS
        ingr_list = soup.find_all('li', class_='wprm-recipe-ingredient')  # Replaced temp_soup with soup
        if ingr_list:
            ingredient_dict = {}
            ingredients_recipe = []
            for ingr in ingr_list:
                ingr_amount_elem = ingr.find(class_='wprm-recipe-ingredient-amount')
                ingr_amount = ingr_amount_elem.text.strip() if ingr_amount_elem else None
                ingr_name_elem = ingr.find(class_='wprm-recipe-ingredient-name')
                ingr_name = ingr_name_elem.text.strip() if ingr_name_elem else None
                ingredient_dict[ingr_name] = ingr_amount if ingr_amount else "seasoning"
                ingredients_recipe.append(ingr_name)
            ingredients_measurment.append(ingredient_dict)
            ingredients.append(ingredients_recipe)
        else:
            ingredients_measurment.append('-')
            ingredients.append('-')

        # INSTRUCTIONS
        recipe_instr = soup.find(class_='wprm-recipe-instruction-group')  # Replaced temp_soup with soup
        if recipe_instr:
            recipe_steps = []
            steps = recipe_instr.find_all("span", attrs={"class": None})
            for step in steps:
                recipe_steps.append(step.text.strip())
            instructions.append(recipe_steps)
        else:
            instructions.append('-')

        # SERVINGS
        serving_div = soup.find('span', class_='wprm-recipe-servings')  # Replaced temp_soup with soup
        if serving_div:
            servings.append(serving_div.text.strip())
        else:
            servings.append('-')

        # CALORIES
        calorie_div = soup.find('span',class_='wprm-recipe-nutrition-with-unit')  # Replaced temp_soup with soup
        if calorie_div:
            calories.append(calorie_div.text.strip())
        else:
            calories.append('-')

        # IMAGES
        images.append('-')
        img_div = soup.find(class_='entry-content')
        img_tags = img_div.find_all('img')
        jpg_links = [img['src'] for img in img_tags if img['src'].endswith('.jpg')]
        if len(jpg_links) >= 2:
            second_jpg_link = jpg_links[1]
            images[-1] = second_jpg_link

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch the page: {e}")
        failed_url.append(url)  # Append failed URL to list


### Recipes 0-24

In [8]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_1 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(0, 150):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = all_recipes_url[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_1['title'] = title
temp_recipes_1['prep_time'] = prep_time
temp_recipes_1['cook_time'] = cook_time
temp_recipes_1['total_time'] = total_time
temp_recipes_1['ingredients'] = ingredients
temp_recipes_1['ingredients with measurement'] = ingredients_measurment
temp_recipes_1['instructions'] = instructions
temp_recipes_1['calories'] = calories
temp_recipes_1['servings'] = servings
temp_recipes_1['images']= images



    

Proxies loaded: ['35.220.254.137:8080', '181.114.61.17:8085']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

Successfully retrieved 147 - https://www.halfbakedharvest.com/tomato-salami-and-chunky-arugula-basil-pesto-bruschetta/ with proxy 181.114.61.17:8085


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully retrieved 148 - https://www.halfbakedharvest.com/quick-balsamic-fig-roasted-tomato-and-burrata-cheese-tarts/ with proxy 35.220.254.137:8080
Proxy 181.114.61.17:8085 failed for https://www.halfbakedharvest.com/kentucky-hot-brown-sandwiches-jalapeno-cornbread-biscuits-honey-bourbon-bacon/: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [9]:
print(len(failed_url))

55


In [11]:
failed_url

['https://www.halfbakedharvest.com/tomato-cheese-pie/',
 'https://www.halfbakedharvest.com/garlic-oil-feta-dip/',
 'https://www.halfbakedharvest.com/fried-goat-cheese-balls/',
 'https://www.halfbakedharvest.com/popular-holiday-appetizers-and-cocktails-mocktails/',
 'https://www.halfbakedharvest.com/beef-tenderloin-crostini/',
 'https://www.halfbakedharvest.com/the-50-most-popular-fall-recipes/',
 'https://www.halfbakedharvest.com/parmesan-buffalo-chicken-wings/',
 'https://www.halfbakedharvest.com/mezze-platter/',
 'https://www.halfbakedharvest.com/jalapeno-cheddar-cornbread-biscuits/',
 'https://www.halfbakedharvest.com/street-corn-dip-sliders/',
 'https://www.halfbakedharvest.com/baked-artichokes/',
 'https://www.halfbakedharvest.com/hot-honey-cauliflower/',
 'https://www.halfbakedharvest.com/chicken-enchilada-bean-dip/',
 'https://www.halfbakedharvest.com/everything-ranch-crackers-and-pretzels/',
 'https://www.halfbakedharvest.com/bacon-wrapped-cheddar-corn-jalapeno-poppers/',
 'htt

In [12]:
temp_recipes_1

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images
0,Sour Cream and Onion Smashed Potatoes,{'minutes': '15'},{'hours': '1'},"{'hours': '1', 'minutes': '15'}","[small to medium potatoes, beer, sea salt and ...","{'small to medium potatoes': '1', 'beer': '1',...","[1. Preheat the oven to 400° F., 2. Arrange th...",136 kcal,6,https://www.halfbakedharvest.com/wp-content/up...
1,Open Face Tomato Bacon Sandwich,{'minutes': '15'},{'minutes': '10'},{'minutes': '25'},"[thick-cut bacon, chopped, extra virgin olive ...","{'thick-cut bacon, chopped': '6', 'extra virgi...",[1. To make the dressing. Combine all ingredie...,641 kcal,6,https://www.halfbakedharvest.com/wp-content/up...
2,Effortless Marinated Burrata,{'minutes': '10'},{'minutes': '10'},{'minutes': '20'},"[marinated mozzarella balls, fresh burrata che...","{'marinated mozzarella balls': '1', 'fresh bur...",[1. Arrange the burrata in a shallow serving b...,372 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
3,Pull Apart Cheeseburger Sliders.,{'minutes': '20'},{'minutes': '30'},{'minutes': '50'},"[mayo, sweet Thai chili sauce, tamari/soy sauc...","{'mayo': '1/2', 'sweet Thai chili sauce': '1/3...",[1. To make the sauce: combine all ingredients...,294 kcal,12,https://www.halfbakedharvest.com/wp-content/up...
4,-,-,-,-,-,-,-,-,-,https://www.halfbakedharvest.com/wp-content/up...
...,...,...,...,...,...,...,...,...,...,...
90,Cheesy Miso Caramelized Corn and Pineapple Chi...,{'minutes': '10'},{'minutes': '25'},{'minutes': '35'},"[butter or olive oil, fresh corn, finely diced...","{'butter or olive oil': '3', 'fresh corn': '4'...",[Add the butter to a large oven-safe skillet s...,548 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
91,Summer Fruit Salsa and Miso Hummus.,{'minutes': '15'},-,{'minutes': '15'},"[cooked chickpeas, tahini, creamy almond butte...","{'cooked chickpeas': '2', 'tahini': '1/4', 'cr...",[],752 kcal,3,https://www.halfbakedharvest.com/wp-content/up...
92,Greek Feta Fries with Roasted Garlic Saffron A...,{'minutes': '15'},{'minutes': '30'},{'minutes': '45'},"[russet potatoes, cut into 1/4 inch wide strip...","{'russet potatoes, cut into 1/4 inch wide stri...",[],677 kcal,4,https://www.halfbakedharvest.com/wp-content/up...
93,"Tomato, Salami and Chunky Arugula-Basil Pesto ...",{'minutes': '10'},{'minutes': '5'},{'minutes': '15'},"[fresh basil, fresh arugula, garlic, roasted c...","{'fresh basil': '2', 'fresh arugula': '1', 'ga...",[],495 kcal,6,https://www.halfbakedharvest.com/wp-content/up...


In [13]:
title

['Sour Cream and Onion Smashed Potatoes',
 'Open Face Tomato Bacon Sandwich',
 'Effortless Marinated Burrata',
 'Pull Apart Cheeseburger Sliders.',
 '-',
 'Watermelon Peach Salad',
 'Granny’s 7 Layer Dip',
 'Honey Garlic Parmesan Biscuits',
 'Pool Day Spicy BBQ Snack Mix.',
 'Roasted Garlic Tomato Caprese',
 '-',
 'Sun-Dried Tomato Garlic Olive Oil Dip',
 'Baked Garlic Parmesan Boneless Wings',
 'Prosciutto Wrapped Baked Brie Rolls',
 'Cranberry Goat Cheese Puff Pastry Swirls',
 'Pastry Wrapped Apple Baked Brie',
 'Garlic Butter Pizza Egg Rolls',
 'Crispy Jalapeño Cream Cheese Cauliflower Taquitos',
 'Tomato Burrata Pepperoni Pizza Dip',
 'Peach Burrata Parmesan Crusted Crostini with Hot Honey',
 'Marinated Cherry Tomatoes with Burrata',
 'Baked Gruyère Tomato Bruschetta',
 'Strawberry Jam with Honey Whipped Goat Cheese',
 'Baked Chipotle Salmon Sushi Cups',
 'Pesto Cheese Garlic Butter Soft Pretzel Knots',
 'Baked Brie Crostini with Crispy Prosciutto',
 'Roasted Garlic Bread Caprese D

In [14]:
final = pd.concat([final, temp_recipes_1], ignore_index=True)

In [15]:
final

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images
0,Sour Cream and Onion Smashed Potatoes,{'minutes': '15'},{'hours': '1'},"{'hours': '1', 'minutes': '15'}","[small to medium potatoes, beer, sea salt and ...","{'small to medium potatoes': '1', 'beer': '1',...","[1. Preheat the oven to 400° F., 2. Arrange th...",136 kcal,6,https://www.halfbakedharvest.com/wp-content/up...
1,Open Face Tomato Bacon Sandwich,{'minutes': '15'},{'minutes': '10'},{'minutes': '25'},"[thick-cut bacon, chopped, extra virgin olive ...","{'thick-cut bacon, chopped': '6', 'extra virgi...",[1. To make the dressing. Combine all ingredie...,641 kcal,6,https://www.halfbakedharvest.com/wp-content/up...
2,Effortless Marinated Burrata,{'minutes': '10'},{'minutes': '10'},{'minutes': '20'},"[marinated mozzarella balls, fresh burrata che...","{'marinated mozzarella balls': '1', 'fresh bur...",[1. Arrange the burrata in a shallow serving b...,372 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
3,Pull Apart Cheeseburger Sliders.,{'minutes': '20'},{'minutes': '30'},{'minutes': '50'},"[mayo, sweet Thai chili sauce, tamari/soy sauc...","{'mayo': '1/2', 'sweet Thai chili sauce': '1/3...",[1. To make the sauce: combine all ingredients...,294 kcal,12,https://www.halfbakedharvest.com/wp-content/up...
4,-,-,-,-,-,-,-,-,-,https://www.halfbakedharvest.com/wp-content/up...
...,...,...,...,...,...,...,...,...,...,...
90,Cheesy Miso Caramelized Corn and Pineapple Chi...,{'minutes': '10'},{'minutes': '25'},{'minutes': '35'},"[butter or olive oil, fresh corn, finely diced...","{'butter or olive oil': '3', 'fresh corn': '4'...",[Add the butter to a large oven-safe skillet s...,548 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
91,Summer Fruit Salsa and Miso Hummus.,{'minutes': '15'},-,{'minutes': '15'},"[cooked chickpeas, tahini, creamy almond butte...","{'cooked chickpeas': '2', 'tahini': '1/4', 'cr...",[],752 kcal,3,https://www.halfbakedharvest.com/wp-content/up...
92,Greek Feta Fries with Roasted Garlic Saffron A...,{'minutes': '15'},{'minutes': '30'},{'minutes': '45'},"[russet potatoes, cut into 1/4 inch wide strip...","{'russet potatoes, cut into 1/4 inch wide stri...",[],677 kcal,4,https://www.halfbakedharvest.com/wp-content/up...
93,"Tomato, Salami and Chunky Arugula-Basil Pesto ...",{'minutes': '10'},{'minutes': '5'},{'minutes': '15'},"[fresh basil, fresh arugula, garlic, roasted c...","{'fresh basil': '2', 'fresh arugula': '1', 'ga...",[],495 kcal,6,https://www.halfbakedharvest.com/wp-content/up...


### Recipes 25-200

In [16]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_2 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(150, 360):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = all_recipes_url[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_2['title'] = title
temp_recipes_2['prep_time'] = prep_time
temp_recipes_2['cook_time'] = cook_time
temp_recipes_2['total_time'] = total_time
temp_recipes_2['ingredients'] = ingredients
temp_recipes_2['ingredients with measurement'] = ingredients_measurment
temp_recipes_2['instructions'] = instructions
temp_recipes_2['calories'] = calories
temp_recipes_2['servings'] = servings
temp_recipes_2['images']= images



    

Proxies loaded: ['35.220.254.137:8080', '181.114.61.17:8085']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully retrieved 150 - https://www.halfbakedharvest.com/cheesy-avocado-crab-stuffed-piquillo-pepper-ciabatta-pizzettes-with-sriracha-aioli/ with proxy 35.220.254.137:8080
Proxy 181.114.61.17:8085 failed for https://www.halfbakedharvest.com/authentic-canadian-poutine/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Max retries exceeded with url: /authentic-canadian-poutine/ (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1374ce9a0>, 'Connection to 181.114.61.17 timed out. (connect timeout=30)')))


...

Successfully retrieved 323 - https://www.halfbakedharvest.com/pull-apart-italian-sub-sliders/ with proxy 181.114.61.17:8085


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully retrieved 324 - https://www.halfbakedharvest.com/charred-peppers/ with proxy 35.220.254.137:8080
Proxy 181.114.61.17:8085 failed for https://www.halfbakedharvest.com/strawberry-watermelon-salad/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Read timed out. (read timeout=30)
Proxy 35.220.254.137:8080 failed for https://www.halfbakedharvest.com/avocado-shrimp-salsa/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Max retries exceeded with url: /avocado-shrimp-salsa/ (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1434b6190>: Failed to establish a new connection: [Errno 51] Network is unreachable')))
Proxy 181.114.61.17:8085 failed for https://www.halfbakedharvest.com/jalapeno-cheddar-popovers/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Max retries exceeded with url: /jalapeno-cheddar-popovers/ (Caused by ProxyError('Unable to connect to proxy', Ne

In [17]:
print(len(failed_url))

157


In [18]:
failed_url

['https://www.halfbakedharvest.com/tomato-cheese-pie/',
 'https://www.halfbakedharvest.com/garlic-oil-feta-dip/',
 'https://www.halfbakedharvest.com/fried-goat-cheese-balls/',
 'https://www.halfbakedharvest.com/popular-holiday-appetizers-and-cocktails-mocktails/',
 'https://www.halfbakedharvest.com/beef-tenderloin-crostini/',
 'https://www.halfbakedharvest.com/the-50-most-popular-fall-recipes/',
 'https://www.halfbakedharvest.com/parmesan-buffalo-chicken-wings/',
 'https://www.halfbakedharvest.com/mezze-platter/',
 'https://www.halfbakedharvest.com/jalapeno-cheddar-cornbread-biscuits/',
 'https://www.halfbakedharvest.com/street-corn-dip-sliders/',
 'https://www.halfbakedharvest.com/baked-artichokes/',
 'https://www.halfbakedharvest.com/hot-honey-cauliflower/',
 'https://www.halfbakedharvest.com/chicken-enchilada-bean-dip/',
 'https://www.halfbakedharvest.com/everything-ranch-crackers-and-pretzels/',
 'https://www.halfbakedharvest.com/bacon-wrapped-cheddar-corn-jalapeno-poppers/',
 'htt

In [19]:
temp_recipes_2

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images
0,Cheesy Avocado Crab Stuffed Piquillo Pepper Ci...,{'minutes': '15'},{'minutes': '20'},{'minutes': '35'},"[egg yolk, apple cider vinegar, juice of 1 lem...","{'egg yolk': '1', 'apple cider vinegar': '1', ...",[],404 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
1,Smoky 3 Cheese Fondue with Toasted Garlic Butt...,{'minutes': '10'},{'minutes': '20'},{'minutes': '30'},"[smoked gouda cheese, gruyere cheese, swiss ch...","{'smoked gouda cheese': '12', 'gruyere cheese'...",[],850 kcal,6,https://www.halfbakedharvest.com/wp-content/up...
2,Greek Feta Chickpea Pancake Fritters with Poac...,{'minutes': '10'},{'minutes': '20'},{'minutes': '30'},"[olive oil, medium leek, salt and pepper, kale...","{'olive oil': '1/4', 'medium leek': '1', 'salt...",[],858 kcal,4,https://www.halfbakedharvest.com/wp-content/up...
3,Fully Loaded Potato Skins with Chipotle Southw...,{'minutes': '10'},"{'hours': '1', 'minutes': '30'}","{'hours': '1', 'minutes': '40'}","[medium size russet potatoes, butter, salt and...","{'medium size russet potatoes': '8', 'butter':...",[],366 kcal,16,https://www.halfbakedharvest.com/wp-content/up...
4,Big Popper Hot Buffalo Chicken and Bacon Grill...,{'minutes': '25'},{'minutes': '20'},{'minutes': '45'},"[boneless skinless chicken breast tenders, egg...",{'boneless skinless chicken breast tenders': '...,[],1028 kcal,4,https://www.halfbakedharvest.com/wp-content/up...
...,...,...,...,...,...,...,...,...,...,...
103,Sweet Cheesy Corn Wontons,{'minutes': '20'},{'minutes': '10'},{'minutes': '30'},"[fresh yellow corn, mayo, shredded mozzarella,...","{'fresh yellow corn': '2', 'mayo': '1/3', 'shr...","[1. In a bowl, toss together the corn, mayo, m...",95 kcal,24,https://www.halfbakedharvest.com/wp-content/up...
104,Tomato Peach Crostini with Hot Bacon Dressing,{'minutes': '15'},{'minutes': '15'},{'minutes': '30'},"[thick-cut slices of bacon, chopped, extra vir...","{'thick-cut slices of bacon, chopped': '6', 'e...",[1. To make the dressing. Combine all ingredie...,468 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
105,Mexican Street Corn Dip,{'minutes': '15'},{'minutes': '15'},{'minutes': '30'},"[chili powder, smoked paprika, cayenne pepper,...","{'chili powder': '2', 'smoked paprika': '2', '...","[1. Mix the spices. In a bowl, combine the chi...",282 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
106,Pull Apart Italian Sub Sliders,{'minutes': '20'},{'minutes': '20'},{'minutes': '45'},"[dinner rolls, halved lengthwise, dijon mustar...","{'dinner rolls, halved lengthwise': '1', 'dijo...","[1. Preheat the oven to 400° F., 2. Place the ...",243 kcal,12,https://www.halfbakedharvest.com/wp-content/up...


In [21]:
final = pd.concat([final, temp_recipes_2], ignore_index=True)
final

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images
0,Sour Cream and Onion Smashed Potatoes,{'minutes': '15'},{'hours': '1'},"{'hours': '1', 'minutes': '15'}","[small to medium potatoes, beer, sea salt and ...","{'small to medium potatoes': '1', 'beer': '1',...","[1. Preheat the oven to 400° F., 2. Arrange th...",136 kcal,6,https://www.halfbakedharvest.com/wp-content/up...
1,Open Face Tomato Bacon Sandwich,{'minutes': '15'},{'minutes': '10'},{'minutes': '25'},"[thick-cut bacon, chopped, extra virgin olive ...","{'thick-cut bacon, chopped': '6', 'extra virgi...",[1. To make the dressing. Combine all ingredie...,641 kcal,6,https://www.halfbakedharvest.com/wp-content/up...
2,Effortless Marinated Burrata,{'minutes': '10'},{'minutes': '10'},{'minutes': '20'},"[marinated mozzarella balls, fresh burrata che...","{'marinated mozzarella balls': '1', 'fresh bur...",[1. Arrange the burrata in a shallow serving b...,372 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
3,Pull Apart Cheeseburger Sliders.,{'minutes': '20'},{'minutes': '30'},{'minutes': '50'},"[mayo, sweet Thai chili sauce, tamari/soy sauc...","{'mayo': '1/2', 'sweet Thai chili sauce': '1/3...",[1. To make the sauce: combine all ingredients...,294 kcal,12,https://www.halfbakedharvest.com/wp-content/up...
4,-,-,-,-,-,-,-,-,-,https://www.halfbakedharvest.com/wp-content/up...
...,...,...,...,...,...,...,...,...,...,...
306,Sweet Cheesy Corn Wontons,{'minutes': '20'},{'minutes': '10'},{'minutes': '30'},"[fresh yellow corn, mayo, shredded mozzarella,...","{'fresh yellow corn': '2', 'mayo': '1/3', 'shr...","[1. In a bowl, toss together the corn, mayo, m...",95 kcal,24,https://www.halfbakedharvest.com/wp-content/up...
307,Tomato Peach Crostini with Hot Bacon Dressing,{'minutes': '15'},{'minutes': '15'},{'minutes': '30'},"[thick-cut slices of bacon, chopped, extra vir...","{'thick-cut slices of bacon, chopped': '6', 'e...",[1. To make the dressing. Combine all ingredie...,468 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
308,Mexican Street Corn Dip,{'minutes': '15'},{'minutes': '15'},{'minutes': '30'},"[chili powder, smoked paprika, cayenne pepper,...","{'chili powder': '2', 'smoked paprika': '2', '...","[1. Mix the spices. In a bowl, combine the chi...",282 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
309,Pull Apart Italian Sub Sliders,{'minutes': '20'},{'minutes': '20'},{'minutes': '45'},"[dinner rolls, halved lengthwise, dijon mustar...","{'dinner rolls, halved lengthwise': '1', 'dijo...","[1. Preheat the oven to 400° F., 2. Place the ...",243 kcal,12,https://www.halfbakedharvest.com/wp-content/up...


In [22]:
def save_list_to_txt(filename, data_list):
    """
    Save a list of strings to a text file, one item per line.
    
    :param filename: The name of the file to save the list to.
    :param data_list: The list of strings to save.
    """
    with open(filename, 'w') as f:
        for item in data_list:
            f.write(f"{item}\n")

In [23]:
save_list_to_txt('failed_urls.txt', failed_url)
final.to_csv('appetizers_1_end.csv', index=False)

### Recipes 201-500

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_3 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(201, 501):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = all_recipes_url[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_3['title'] = title
temp_recipes_3['prep_time'] = prep_time
temp_recipes_3['cook_time'] = cook_time
temp_recipes_3['total_time'] = total_time
temp_recipes_3['ingredients'] = ingredients
temp_recipes_3['ingredients with measurement'] = ingredients_measurment
temp_recipes_3['instructions'] = instructions
temp_recipes_3['calories'] = calories
temp_recipes_3['servings'] = servings
temp_recipes_3['images']= images



    

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_3

In [ ]:
final = pd.concat([final, temp_recipes_3], ignore_index=True)
final

In [ ]:
def save_list_to_txt(filename, data_list):
    """
    Save a list of strings to a text file, one item per line.
    
    :param filename: The name of the file to save the list to.
    :param data_list: The list of strings to save.
    """
    with open(filename, 'w') as f:
        for item in data_list:
            f.write(f"{item}\n")

In [ ]:
save_list_to_txt('failed_urls_till501.txt', failed_url)
final.to_csv('desserts_201_501.csv', index=False)

### Recipes 501-800

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_4 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(501, 792):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = all_recipes_url[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_4['title'] = title
temp_recipes_4['prep_time'] = prep_time
temp_recipes_4['cook_time'] = cook_time
temp_recipes_4['total_time'] = total_time
temp_recipes_4['ingredients'] = ingredients
temp_recipes_4['ingredients with measurement'] = ingredients_measurment
temp_recipes_4['instructions'] = instructions
temp_recipes_4['calories'] = calories
temp_recipes_4['servings'] = servings
temp_recipes_4['images']= images



    

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_4

In [ ]:
final = pd.concat([final, temp_recipes_4], ignore_index=True)
final

In [ ]:
save_list_to_txt('failed_urls_till201_792.txt', failed_url)
final.to_csv('recipes_501_792.csv', index=False)

In [ ]:
urls2 = []

with open('all_urls.txt', 'r') as file:
    for line in file:
        urls2.append(line.strip())

print(urls2)

In [ ]:
failed_url = []

### Recipes 801-1001

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_5 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(801, 1001):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = urls2[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_5['title'] = title
temp_recipes_5['prep_time'] = prep_time
temp_recipes_5['cook_time'] = cook_time
temp_recipes_5['total_time'] = total_time
temp_recipes_5['ingredients'] = ingredients
temp_recipes_5['ingredients with measurement'] = ingredients_measurment
temp_recipes_5['instructions'] = instructions
temp_recipes_5['calories'] = calories
temp_recipes_5['servings'] = servings
temp_recipes_5['images']= images



    

In [ ]:
final3 = pd.DataFrame(columns=columns)
final3

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_5

In [ ]:
final3 = pd.concat([final3, temp_recipes_5], ignore_index=True)
final3

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_6 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(1001, 1401):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = urls2[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_6['title'] = title
temp_recipes_6['prep_time'] = prep_time
temp_recipes_6['cook_time'] = cook_time
temp_recipes_6['total_time'] = total_time
temp_recipes_6['ingredients'] = ingredients
temp_recipes_6['ingredients with measurement'] = ingredients_measurment
temp_recipes_6['instructions'] = instructions
temp_recipes_6['calories'] = calories
temp_recipes_6['servings'] = servings
temp_recipes_6['images']= images



    

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_6

In [ ]:
final3 = pd.concat([final3, temp_recipes_6], ignore_index=True)
final3

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_7 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(1401, 1607):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = urls2[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_7['title'] = title
temp_recipes_7['prep_time'] = prep_time
temp_recipes_7['cook_time'] = cook_time
temp_recipes_7['total_time'] = total_time
temp_recipes_7['ingredients'] = ingredients
temp_recipes_7['ingredients with measurement'] = ingredients_measurment
temp_recipes_7['instructions'] = instructions
temp_recipes_7['calories'] = calories
temp_recipes_7['servings'] = servings
temp_recipes_7['images']= images



    

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_7